In [ ]:
import pandas as pd
import math
import numpy as np
import scipy.io
import scipy.sparse

In [ ]:
hu_aa_p1_barcode = pd.read_csv('../../data/Huetal2022/AA_patient_1/GSM5515741_AA1_barcodes.tsv', sep='\t', header=None)
hu_aa_p1_features = pd.read_csv('../../data/Huetal2022/AA_patient_1/GSM5515741_AA1_features.tsv', sep='\t', header=None)
# The data matrix rows are genes and columns are cells. The indices match the indices of the files imported above. Matries are sparse (common in scRNAseq due to dropout)
hu_aa_p1_data_mtx = scipy.io.mmread('../../data/Huetal2022/AA_patient_1/GSM5515741_AA1_matrix.mtx')
hu_aa_p1_data_df = pd.DataFrame.sparse.from_spmatrix(hu_aa_p1_data_mtx)
hu_aa_p1_data = { "cells": hu_aa_p1_barcode, "genes": hu_aa_p1_features, "data":hu_aa_p1_data_df}
hu_aa_p2_barcode = pd.read_csv('../../data/Huetal2022/AA_patient_2/GSM5515742_AA2_barcodes.tsv', sep='\t', header=None)
hu_aa_p2_features = pd.read_csv('../../data/Huetal2022/AA_patient_2/GSM5515742_AA2_features.tsv', sep='\t', header=None)
hu_aa_p2_data_mtx = scipy.io.mmread('../../data/Huetal2022/AA_patient_2/GSM5515742_AA2_matrix.mtx')
hu_aa_p2_data_df = pd.DataFrame.sparse.from_spmatrix(hu_aa_p2_data_mtx)
hu_aa_p2_data = { "cells": hu_aa_p2_barcode, "genes": hu_aa_p2_features, "data":hu_aa_p2_data_df}
hu_n_p1_barcode = pd.read_csv('../../data/Huetal2022/N_patient_1/GSM5515743_Normal1_barcodes.tsv', sep='\t', header=None)
hu_n_p1_features = pd.read_csv('../../data/Huetal2022/N_patient_1/GSM5515743_Normal1_features.tsv', sep='\t', header=None)
hu_n_p1_data_mtx = scipy.io.mmread('../../data/Huetal2022/N_patient_1/GSM5515743_Normal1_matrix.mtx')
hu_n_p1_data_df = pd.DataFrame.sparse.from_spmatrix(hu_n_p1_data_mtx)
hu_n_p1_data = { "cells": hu_n_p1_barcode, "genes": hu_n_p1_features, "data":hu_n_p1_data_df}
hu_n_p2_barcode = pd.read_csv('../../data/Huetal2022/N_patient_2/GSM5515744_Normal2_barcodes.tsv', sep='\t', header=None)
hu_n_p2_features = pd.read_csv('../../data/Huetal2022/N_patient_2/GSM5515744_Normal2_features.tsv', sep='\t', header=None)
hu_n_p2_data_mtx = scipy.io.mmread('../../data/Huetal2022/N_patient_2/GSM5515744_Normal2_matrix.mtx')
hu_n_p2_data_df = pd.DataFrame.sparse.from_spmatrix(hu_n_p2_data_mtx)
hu_n_p2_data = { "cells": hu_n_p2_barcode, "genes": hu_n_p2_features, "data":hu_n_p2_data_df}

Add col for disease status

In [ ]:
# genes as cols, cells as rows
hu_aa_p1_data_df = hu_aa_p1_data_df.T
hu_aa_p2_data_df = hu_aa_p2_data_df.T
hu_n_p1_data_df = hu_n_p1_data_df.T
hu_n_p2_data_df = hu_n_p2_data_df.T

In [ ]:
# add col for disease status
hu_aa_p1_data_df['Aplastic Anemia'] = 1
hu_aa_p2_data_df['Aplastic Anemia'] = 1
hu_n_p1_data_df['Aplastic Anemia'] = 0
hu_n_p2_data_df['Aplastic Anemia'] = 0

In [ ]:
full_dataset = pd.concat([hu_aa_p1_data_df, hu_aa_p2_data_df, hu_n_p1_data_df, hu_n_p2_data_df])
full_dataset

Split test and training data

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
labels = full_dataset['Aplastic Anemia']

In [ ]:
full_data_no_labels = full_dataset.drop('Aplastic Anemia', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(full_data_no_labels, labels, test_size=0.20)

PCA

In [ ]:
# find singular values
from scipy.linalg import svd

In [ ]:
# SVD
U, s, VT = svd(X_train)

In [ ]:
# save to not have to run again in the future
U.tofile('trainingU.csv', sep=',')

In [ ]:
s.tofile('trainings.csv', sep=',')

In [ ]:
VT.tofile('trainingVT.csv', sep=',')

In [ ]:
X_train_indices = X_train.index
type(list(X_train_indices))
with open("X_train_idxs.txt", "w") as output:
    output.write(str(list(X_train_indices)))

In [ ]:
y_train.to_csv("ytrain.csv")

In [ ]:
#Determine number of modes to incorperate
energyThreshold = .9
normalized_eigvals = s / sum(s)
normalized_eigvals

In [ ]:
cumEnergy = np.cumsum(s)/sum(s)
cumEnergy

In [ ]:
energyIndex = np.argwhere(cumEnergy > energyThreshold)
energyIndex = int(min(energyIndex))
energyIndex # use rank 1939 to get 90% of energy

In [ ]:
# project onto determined number or modes
rankVT = VT[:, 1:energyIndex]
rankVT.shape

In [ ]:
X_test.shape

In [ ]:
sample = rankVT.T.dot(X_test.T)
training = rankVT.T.dot(X_train.T)

In [ ]:
type(y_train)

In [ ]:
# Classify using LDA
group = y_train
group

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:

clf = LDA()
clf.fit(training.T, group)
LDA(priors=None, shrinkage=None, solver='svd',
  store_covariance=False, tol=0.0001)

In [ ]:
y_test_predictions = clf.predict(sample.T)

In [ ]:
y_test_predictions

In [ ]:
y_test.to_numpy()

In [ ]:
num_correct_predictions = sum(y_test_predictions == y_test.to_numpy())
num_correct_predictions

In [ ]:
num_incorrect_predictions = len(y_test) - num_correct_predictions
num_incorrect_predictions

In [ ]:
percent_correct = num_correct_predictions/len(y_test)
percent_correct

In [ ]:
# ~95% success rate on this test set, lets try on training data
y_training_predictions = clf.predict(training.T)

In [ ]:
num_correct_predictions = sum(y_training_predictions == group.to_numpy())
num_correct_predictions

In [ ]:
num_incorrect_predictions = len(group.to_numpy()) - num_correct_predictions
num_incorrect_predictions

In [ ]:
percent_correct = num_correct_predictions/len(group.to_numpy())
percent_correct

In [ ]:
# about 97% correct
type(X_train)

In [ ]:
# now that we know the number of components to use, lets try just an LDA call from the start
clf2 = LDA()
clf2.fit(X_train.to_numpy(), y_train)
LDA(priors=None, shrinkage=None, solver='svd',
  store_covariance=False, tol=0.0001, n_components = energyIndex)

In [ ]:
training_predictions2 = clf2.predict(X_train)


In [ ]:
test_predictions2 = clf2.predict(X_test)

In [ ]:
num_correct_train_predictions_2 = sum(training_predictions2 == y_train.to_numpy())
percent_correct = num_correct_train_predictions_2/len(y_train)
percent_correct

In [ ]:
num_correct_test_predictions_2 = sum(test_predictions2 == y_test.to_numpy())
percent_correct = num_correct_test_predictions_2/len(y_test)
percent_correct

In [ ]:
# 57% that makes a lot more sense

In [ ]:
energyIndex

In [ ]:
hu_aa_p1_data_mtx